In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import ccxt
import csv
import json
import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
from pathlib import Path
from IPython import get_ipython

exchange = ccxt.ftx({
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True
    }
})
_ =exchange.loadMarkets()
milliseconds = {key: int(value) * 1000 for key, value in exchange.timeframes.items()}
milliseconds

{'15s': 15000,
 '1m': 60000,
 '5m': 300000,
 '15m': 900000,
 '1h': 3600000,
 '4h': 14400000,
 '1d': 86400000}

In [30]:
def merge(path, data):
    df = pd.read_csv(path, header=0)
    headers = df.columns.tolist()

    # Incoming data has no column headers
    df_to_merge = pd.DataFrame(data, columns=headers)
    df = pd.concat([df, df_to_merge]) \
        .astype({'timestamp': 'int64'}) \
        .drop_duplicates('timestamp') \
        .sort_values('timestamp') \
        .reset_index(drop=True)
    
    min1 = df['timestamp'].min()
    print(min1 not in df['timestamp'].values)
    return df


fullpath1 = Path('../data/1.csv')
fullpath2 = Path('../data/2.csv')

merge(fullpath1, [['159394710000', '9021.5', '9030.0', '9021.0', '9023.0', '419473.44695aaa']])


False


,timestamp,open,high,low,close,volume
0,159394710000,9021.5,9030.0,9021.0,9023.0,419473.44695aaa
1,1593945900000,9036.5,9041,9034.5,9034.5,277605
2,1593946200000,9034.5,9034.5,9027,9028,174219
3,1593946500000,9028,9033,9027.5,9027.5,274874
4,1593946800000,9027.5,9036.5,9027.5,9034,339279
5,1593947100000,9035,9043,9034.5,9040.5,224655
6,1593947400000,9040.5,9042,9037,9038.5,150016
7,1593947700000,9038.5,9038.5,9036.5,9036.5,51065.1
8,1593948000000,9036.5,9037,9036.5,9037,20800.4
9,1593948300000,9037,9043,9037,9040,136009


In [2]:

class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        # keep a reference to the "close" line in the data[0] dataseries?
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/sell order submitted/accepted to/by broker- nothing to do
            return

        # Check if order has been completed. Broker can reject if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm: %2f' % 
                    (order.executed.price, 
                    order.executed.value, 
                    order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %2f' % 
                    (order.executed.price,
                    order.executed.value,
                    order.executed.comm))
                
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order canceled/margin/rejected')
        
        # write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % 
            (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the references
        self.log('Close, %.2f' % (self.dataclose[0]))
    
        # if order pending, cannot second a second one
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:

            if self.dataclose[0] < self.dataclose[-1]:
                # current close less than previous close
                if self.dataclose[-1] < self.dataclose[-2]:
                    # previous close less than previous previous close
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    availCash = self.broker.getcash()
                    currentClose = self.dataclose[0]
                    units = (availCash / currentClose) * 0.9
                    self.order = self.buy(size=units)
        else:
            #already in the market... we might sell
            if len(self) >= (self.bar_executed + 5):
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.close()

In [5]:
cerebro = bt.Cerebro(stdstats=False)
cerebro.addstrategy(TestStrategy)
cerebro.addobserver(bt.observers.BuySell, barplot=True, bardist=0.005)
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)

p = Path('../data/')
fullPath = p / 'ftx-BTC-PERP-5m.csv'
data = btfeeds.GenericCSVData(
    dataname=fullPath,
    timeframe=bt.TimeFrame.Minutes, compression=5,
    dtformat=lambda x: datetime.datetime.utcfromtimestamp(int(x) // 1000),
    openinterest=-1
)

cerebro.adddata(data)
cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.0007)

print('Starting portfolio value: %.2f' % (cerebro.broker.getvalue()))

cerebro.run()

print('Final value: %.2f' % (cerebro.broker.getvalue()))


-06-10T09:50:00, Close, 37415.00
2021-06-10T09:55:00, Close, 37458.00
2021-06-10T10:00:00, Close, 38295.00
2021-06-10T10:05:00, Close, 38230.00
2021-06-10T10:10:00, Close, 38149.00
2021-06-10T10:10:00, BUY CREATE, 38149.00
2021-06-10T10:15:00, BUY EXECUTED, Price: 38149.00, Cost: 1.09, Comm: 0.000761
2021-06-10T10:15:00, Close, 38253.00
2021-06-10T10:20:00, Close, 38218.00
2021-06-10T10:25:00, Close, 38177.00
2021-06-10T10:30:00, Close, 38214.00
2021-06-10T10:35:00, Close, 38193.00
2021-06-10T10:40:00, Close, 38115.00
2021-06-10T10:40:00, SELL CREATE, 38115.00
2021-06-10T10:45:00, SELL EXECUTED, Price: 38115.00, Cost: 1.09, Comm: 0.000761
2021-06-10T10:45:00, OPERATION PROFIT, GROSS -0.00, NET -0.00
2021-06-10T10:45:00, Close, 37940.00
2021-06-10T10:45:00, BUY CREATE, 37940.00
2021-06-10T10:50:00, BUY EXECUTED, Price: 37940.00, Cost: 1.09, Comm: 0.000760
2021-06-10T10:50:00, Close, 37943.00
2021-06-10T10:55:00, Close, 37862.00
2021-06-10T11:00:00, Close, 37901.00
2021-06-10T11:05:00, C

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 12]
plt.rcParams.update({'font.size': 12})
cerebro.plot(iplot=False, style='candlestick')